<a href="https://colab.research.google.com/github/squashai/notebooks/blob/main/positions_superposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Superposition of court positions of squash players in random videos

This notebook extracts player positions from a squash video using the convolutional neural network created and trained in [baseline-court-positions](https://github.com/squashai/notebooks/blob/main/baseline_court_positions.ipynb), and assembles a video with such positions superposed to the original one.

[![SQUASHAI](https://img.shields.io/badge/SQUASH-AI-black)](https://squashai.github.io)
[![View source on Github](https://img.shields.io/badge/-View%20source%20on%20Github-blue?logo=github&labelColor=black)](https://github.com/squashai/notebooks/blob/main/positions_superposition.ipynb)


# Imports and Variables

In [ ]:
import cv2
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from keras.models import load_model

In [ ]:
VIDEO_URL = 'https://www.dropbox.com/s/f79eakp3u2lvo5e/squash-rally.mp4?dl=1'
MODEL_URL = 'https://github.com/squashai/notebooks/releases/download/0.0.1/best_bl-positions_model.hdf5'
BACKGROUND_URL = 'https://raw.githubusercontent.com/squashai/squashai/master/src/assets/ground.png'

VIDEO_FILE = 'squash-rally.mp4'
MODEL_FILE = 'best_bl-positions_model.hdf5'
BACKGROUND_FILE = 'ground.png'
OUTPUT_FILE = 'enanched-squash-rally.mp4'

MODEL_INPUT_SIZE = (108, 192)

COURT_SIZE = (195, 128)

OUTPUT_OVERLAY_OFFSET = (20, 20)
OUTPUT_OVERLAY_SIZE = (487, 320)

# Data Retrivial

In [ ]:
%cd /content/
!curl -L --output $VIDEO_FILE $VIDEO_URL
!curl -L --output $MODEL_FILE $MODEL_URL
!curl -L --output $BACKGROUND_FILE $BACKGROUND_URL

# Prediction and Output Video Composition

In [ ]:
model = load_model(MODEL_FILE)

In [ ]:
def scale_image(image, output_size):
    h, w = image.shape[:2]
    if isinstance(output_size, int):
        if h > w:
            new_h, new_w = output_size * h / w, output_size
        else:
            new_h, new_w = output_size, output_size * w / h
    else:
        new_h, new_w = output_size

    new_h, new_w = int(new_h), int(new_w)
    new_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)
    return new_image

In [ ]:
background = np.array(mpimg.imread(BACKGROUND_FILE))
#background = cv2.resize(bg, COURT_SIZE[::-1], interpolation=cv2.INTER_AREA)
background = scale_image(background, COURT_SIZE)

def prediction_to_frame(prediction):

    overlay = prediction[0, :COURT_SIZE[0], :COURT_SIZE[1], 0]
    overlay -= np.min(overlay)
    overlay /= np.max(overlay)

    frame = np.array(background[:, :, :3])
    frame[:, :, 0] = np.maximum(frame[:, :, 0] * (1 - overlay), overlay)
    frame[:, :, 1] = np.maximum(frame[:, :, 1] * (1 - overlay), overlay)
    #frame[:, :, 2] = np.maximum(frame[:, :, 2] * (1 - overlay), overlay)

    return frame

In [ ]:
source_video = cv2.VideoCapture(VIDEO_FILE)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
framerate = source_video.get(cv2.CAP_PROP_FPS)
size = (int(source_video.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(source_video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
destination_video = cv2.VideoWriter(OUTPUT_FILE, fourcc, framerate, size)

success, source_frame = source_video.read()
while success:
    model_input = scale_image(source_frame, MODEL_INPUT_SIZE)
    model_input = np.array([cv2.cvtColor(model_input, cv2.COLOR_BGR2RGB)])
    model_prediction = model.predict(model_input)
    court_frame = prediction_to_frame(model_prediction)

    court_frame = scale_image(court_frame, OUTPUT_OVERLAY_SIZE)
    x_min = OUTPUT_OVERLAY_OFFSET[0]
    x_max = OUTPUT_OVERLAY_OFFSET[0] + court_frame.shape[0]
    y_min = OUTPUT_OVERLAY_OFFSET[1]
    y_max = OUTPUT_OVERLAY_OFFSET[1] + court_frame.shape[1]

    destination_frame = cv2.cvtColor(source_frame, cv2.COLOR_BGR2RGB)
    court_frame = np.array(court_frame * 255, dtype=np.uint8)
    destination_frame[x_min:x_max, y_min:y_max, :] = court_frame[:, :, :]

    destination_video.write(cv2.cvtColor(destination_frame, cv2.COLOR_RGB2BGR))

    success, source_frame = source_video.read()

source_video.release()
destination_video.release()

In [ ]:
!cp $OUTPUT_FILE /content/drive/My\ Drive/